In [3]:
from pydantic import BaseModel, ConfigDict, Field, ValidationError

In [4]:
class Point2D(BaseModel):
    x: float = 0
    y: float = 0

In [5]:
class Circle2D(BaseModel):
    center: Point2D
    radius: float = Field(default=1, gt=0)

In [6]:
c = Circle2D(center=Point2D(x=1, y=1))

In [7]:
c

Circle2D(center=Point2D(x=1.0, y=1.0), radius=1)

In [8]:
c.model_dump()

{'center': {'x': 1.0, 'y': 1.0}, 'radius': 1}

In [9]:
c.model_dump_json()

'{"center":{"x":1.0,"y":1.0},"radius":1.0}'

In [10]:
data = {
    "center":{
        "x": 5,
        "y": 5
    },
    "radius": 2

}

In [11]:
Circle2D.model_validate(data)
# Similar for JSON

Circle2D(center=Point2D(x=5.0, y=5.0), radius=2.0)

In [12]:
json_data = """
{
    "firstName": "David",
    "lastName": "Hilbert",
    "contactInfo": {
        "email": "d.hilbert@spectral-theory.com",
        "homePhone": {
            "countryCode": 49,
            "areaCode": 551,
            "localPhoneNumber": 123456789
        }
    },
    "personalInfo": {
        "nationality": "German",
        "born": {
            "date": "1862-01-23",
            "place": {
                "city": "Konigsberg",
                "country": "Prussia"
            }
        },
        "died": {
            "date": "1943-02-14",
            "place": {
                "city": "Gottingen",
                "country": "Germany"
            }
        }
    },
    "awards": ["Lobachevsky Prize", "Bolyai Prize", "ForMemRS"],
    "notableStudents": ["von Neumann", "Weyl", "Courant", "Zermelo"]
}
"""

In [13]:
from pydantic import EmailStr, PastDate
from pydantic.alias_generators import to_camel

In [14]:
class ContactInfo(BaseModel):
    model_config = ConfigDict(extra="ignore")

    emai: EmailStr | None = None

class PlaceInfo(BaseModel):
    city: str
    country: str

class PlaceDateInfo(BaseModel):
    model_config = ConfigDict(populate_by_name=True)

    date_: PastDate = Field(alias="date")
    place: PlaceInfo

class PersonalInfo(BaseModel):
    model_config = ConfigDict(extra="ignore")

class Person(BaseModel):
    model_config = ConfigDict(alias_generator=to_camel, populate_by_name=True, extra="ignore")

    first_name: str
    last_name: str
    contact_info: ContactInfo
    personal_info: PersonalInfo
    notable_students: list[str] = []

In [15]:
p = Person.model_validate_json(json_data)

In [16]:
p

Person(first_name='David', last_name='Hilbert', contact_info=ContactInfo(emai=None), personal_info=PersonalInfo(), notable_students=['von Neumann', 'Weyl', 'Courant', 'Zermelo'])

In [17]:
print(p.model_dump_json(indent=2, by_alias=True))

{
  "firstName": "David",
  "lastName": "Hilbert",
  "contactInfo": {
    "emai": null
  },
  "personalInfo": {},
  "notableStudents": [
    "von Neumann",
    "Weyl",
    "Courant",
    "Zermelo"
  ]
}


In [18]:
from typing import Annotated
from pydantic import AfterValidator

SortedStringList = Annotated[list[str], AfterValidator(lambda value: sorted(value, key=str.casefold))]

class Person(BaseModel):
    model_config = ConfigDict(alias_generator=to_camel, populate_by_name=True, extra="ignore")

    first_name: str
    last_name: str
    contact_info: ContactInfo
    personal_info: PersonalInfo
    notable_students: SortedStringList = []

In [19]:
p = Person.model_validate_json(json_data)
p

Person(first_name='David', last_name='Hilbert', contact_info=ContactInfo(emai=None), personal_info=PersonalInfo(), notable_students=['Courant', 'von Neumann', 'Weyl', 'Zermelo'])

In [20]:
print(p.model_dump_json(by_alias=True, indent=2))

{
  "firstName": "David",
  "lastName": "Hilbert",
  "contactInfo": {
    "emai": null
  },
  "personalInfo": {},
  "notableStudents": [
    "Courant",
    "von Neumann",
    "Weyl",
    "Zermelo"
  ]
}
